# 0. Imports

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_raw = pd.read_csv('train.csv')

# 1. Data Description

In [4]:
df1 = df_raw.copy()

## 1.1 Data Cleaning

In [5]:
# Setting columns to lower
df1.columns = [i.lower() for i in df1.columns ]

# Removing N/A and converting to int - delivery_person_age'
lines = df1['delivery_person_age'] != 'NaN '
df1 = df1.loc[lines, :].copy()
df1['delivery_person_age'] = df1['delivery_person_age'].astype('int64')

# Removing N/A - road_traffic_density
lines = df1['road_traffic_density'] != 'NaN '
df1 = df1.loc[lines, :].copy()

# Removing N/A - festival
lines = df1['festival'] != 'NaN '
df1 = df1.loc[lines, :].copy()

# Removing N/A - road_traffic_density
lines = df1['city'] != 'NaN '
df1 = df1.loc[lines, :].copy()

# Transforming time taken in int
df1['time_taken(min)'] = df1['time_taken(min)'].str[-2:].astype('int64')

# Converting to float and replacing N/A with "" - delivery_person_ratings
df1['delivery_person_ratings'].fillna("", inplace=True)
df1['delivery_person_ratings'] = df1['delivery_person_ratings'].astype(float)

# Converting to datetime - order_date
df1['order_date'] = pd.to_datetime(df1['order_date'], format='%d-%m-%Y')

# Removing N/A and converting to int - multiple_deliveries
lines = df1['multiple_deliveries'] != 'NaN '
df1 = df1.loc[lines, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype('int64')

# Removing spaces in object features
df1 = df1.applymap(lambda x: x.strip() if isinstance(x, str) else x)


# 2. Business Insights (EDA)

In [6]:
df2 = df1.copy()

## 2.1 Business view

### 2.1.1 Quantity orders per day

In [7]:
df_aux = df2.groupby('order_date')['id'].count().reset_index(name='count')
df_aux
plt.figure(figsize=(10,8))
px.bar(df_aux, x='order_date', y='count')

<Figure size 1000x800 with 0 Axes>

### 2.1.2 Quantity orders per week

In [8]:
df2['week_of_year'] = df2['order_date'].dt.strftime('%U')
df_aux = df2.groupby('week_of_year')['id'].count().reset_index()
px.line(df_aux, 'week_of_year', 'id')

### 2.1.3 Orders distribution per traffic

In [9]:
df_aux = df2.groupby('road_traffic_density')['id'].count().reset_index()
df_aux = df_aux.loc[df_aux['road_traffic_density'] != 'NaN', :]
df_aux['percentage'] = df_aux['id'] / df_aux['id'].sum() * 100

px.pie(df_aux, names= 'road_traffic_density',values='percentage')

### 2.1.4 Comparison of order volume by city and traffic

In [10]:
df_aux = df2.groupby(['city', 'road_traffic_density'])['id'].count().reset_index()
df_aux = df_aux.loc[df_aux['road_traffic_density'] != "NaN",:]
df_aux = df_aux.loc[df_aux['city'] != "NaN",:]

px.scatter(df_aux, x='city', y='road_traffic_density', size='id', color='city')

### 2.1.5 Deliveries quantity by deliverer person per week

In [11]:
df_aux01 = df2[['id', 'week_of_year']].groupby('week_of_year').count().reset_index()
df_aux02 = df2[['week_of_year', 'delivery_person_id']].groupby('week_of_year').nunique().reset_index()
df_aux = pd.merge(df_aux01, df_aux02, how='inner')
df_aux['deliveries_per_person'] = df_aux['id'] / df_aux['delivery_person_id']

px.line(df_aux, x='week_of_year', y='deliveries_per_person')

### 2.1.6 Central localization of each city by traffic 

In [12]:
import folium
df_aux = df2[['city', 'road_traffic_density', 'delivery_location_latitude', 'delivery_location_longitude']].groupby(['city', 'road_traffic_density']).median().reset_index()

df_aux = df_aux.loc[df_aux['road_traffic_density'] != "NaN",:]
df_aux = df_aux.loc[df_aux['city'] != "NaN",:]

df_aux = df_aux.head()

map = folium.Map()

for index, i in df_aux.iterrows():
    folium.Marker([i['delivery_location_latitude'], i['delivery_location_longitude']], popup= i[['city', 'road_traffic_density']]).add_to(map)

map

## 2.2 Delivery Person View

### 2.2.1 The youngest and oldest age of the delivery people.

In [13]:
print('The youngest delivery person is {} years old'.format(df2['delivery_person_age'].min()))
print('The oldest delivery person is {} years old'.format(df2['delivery_person_age'].max()))

The youngest delivery person is 20 years old
The oldest delivery person is 39 years old


### 2.2.2 The worst and best condition of vehicles.

In [14]:
print('The best vehicle condition is {}'.format(df2['vehicle_condition'].max()))
print('The worst vehicle condition is {}'.format(df2['vehicle_condition'].min()))

The best vehicle condition is 2
The worst vehicle condition is 0


In [15]:
df2['delivery_person_ratings'].unique()

array([4.9, 4.5, 4.4, 4.7, 4.6, 4.8, 4.2, 4.3, 4. , 4.1, 5. , 3.5, 3.8,
       nan, 3.9, 3.7, 2.6, 2.5, 3.6, 3.1, 2.7, 3.2, 3.3, 3.4, 2.8, 2.9,
       3. ])

### 2.2.3 The average rating per delivery person.

In [30]:
df_aux = df2[['delivery_person_ratings', 'delivery_person_id']].groupby('delivery_person_id').mean().reset_index()
df_aux

,delivery_person_id,delivery_person_ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000
...,...,...
1315,VADRES19DEL02,4.632727
1316,VADRES19DEL03,4.670270
1317,VADRES20DEL01,4.620370
1318,VADRES20DEL02,4.591111


### 2.2.4 The average rating and standard deviation by traffic type.

In [34]:
df_aux = df2[['delivery_person_ratings', 'road_traffic_density']].groupby('road_traffic_density').agg({'delivery_person_ratings': ['mean', 'std']})
df_aux.columns = ['rating_mean', 'rating_std']
df_aux = df_aux.reset_index()
df_aux

,road_traffic_density,rating_mean,rating_std
0,High,4.652230,0.273044
1,Jam,4.594019,0.329778
2,Low,4.645011,0.338080
3,Medium,4.660138,0.274245


### 2.2.5 The average rating and standard deviation by climatic conditions.

In [37]:
df2['weatherconditions'].unique()

array(['conditions Sunny', 'conditions Stormy', 'conditions Sandstorms',
       'conditions Cloudy', 'conditions Fog', 'conditions Windy'],
      dtype=object)

In [18]:
df_aux = df2[['delivery_person_ratings', 'weatherconditions']].groupby('weatherconditions').agg({'delivery_person_ratings': ['mean', 'std']})
df_aux.columns = ['rating_mean', 'rating_std']
df_aux = df_aux.reset_index()
df_aux

,weatherconditions,rating_mean,rating_std
0,conditions Cloudy,4.651871,0.281197
1,conditions Fog,4.652965,0.275060
2,conditions Sandstorms,4.611748,0.310852
3,conditions Stormy,4.611819,0.313096
4,conditions Sunny,4.654868,0.396674
5,conditions Windy,4.616128,0.304565


### 2.2.6 Top 10 fastest delivery person by city

In [19]:
df_aux = (df2[['delivery_person_id', 'time_taken(min)', 'city']].groupby(['city', 'delivery_person_id'])
                                                                .mean()
                                                                .sort_values('time_taken(min)')
                                                                .groupby('city')
                                                                .head(10)
                                                                .reset_index()
                                                                )
df_aux


,city,delivery_person_id,time_taken(min)
0,Urban,LUDHRES01DEL02,10.000000
1,Urban,BHPRES11DEL03,10.000000
2,Urban,AURGRES02DEL03,10.000000
3,Urban,KOCRES04DEL02,10.000000
4,Urban,DEHRES08DEL02,10.000000
5,Urban,DEHRES20DEL03,10.000000
6,Urban,LUDHRES07DEL03,11.000000
7,Urban,BHPRES13DEL01,11.000000
8,Urban,KOLRES05DEL01,11.000000
9,Urban,CHENRES07DEL01,11.666667


### 2.2.7 Top 10 slowest delivery person by city

In [20]:
df_aux = (df2[['delivery_person_id', 'time_taken(min)', 'city']].groupby(['city', 'delivery_person_id'])
                                                                .mean()
                                                                .sort_values(['city','time_taken(min)'], ascending=False)
                                                                .groupby('city')
                                                                .head(10)
                                                                .reset_index()
                                                                )
df_aux

,city,delivery_person_id,time_taken(min)
0,Urban,KOLRES19DEL03,51.000000
1,Urban,LUDHRES16DEL03,47.000000
2,Urban,AURGRES17DEL03,43.000000
3,Urban,BHPRES03DEL02,42.000000
4,Urban,DEHRES02DEL03,41.000000
5,Urban,DEHRES09DEL02,39.500000
6,Urban,BHPRES07DEL03,39.333333
7,Urban,GOARES01DEL02,38.500000
8,Urban,AURGRES04DEL01,38.000000
9,Urban,GOARES03DEL01,38.000000


## 2.3 Restaurant View

### 2.3.1 The number of unique delivery people.

In [21]:
df_aux = df2['delivery_person_id'].nunique()
print(f'The number of unique delivery people is {df_aux}')

The number of unique delivery people is 1320


### 2.3.2 The average distance to restaurants and delivery locations.

In [22]:
from haversine import haversine

df2['distance']= df2.apply(lambda x: haversine((x['restaurant_latitude'], x['restaurant_longitude'] ), 
                                              (x['delivery_location_latitude'], x['delivery_location_longitude'])), axis=1)
distance_mean = df2['distance'].mean().round(2)

print('The avarage distance of restaurants and delivery location is {} Km'.format(distance_mean))

The avarage distance of restaurants and delivery location is 27.44 Km


In [47]:
df_aux = df2[['distance', 'city']].groupby('city').mean().reset_index()
px.pie(df_aux, names='city', values= 'distance')

### 2.3.3 Average delivery time and standard deviation by city.

In [51]:
df_aux = df2[['time_taken(min)', 'city']].groupby('city').agg({'time_taken(min)': ['mean', 'std']})
df_aux.columns = ['time_mean', 'time_std']
df_aux = df_aux.reset_index()
df_aux

,city,time_mean,time_std
0,Metropolitian,27.428083,9.133374
1,Semi-Urban,49.710526,2.724992
2,Urban,23.209379,8.858049


In [52]:
fig = go.Figure()

fig.add_trace(go.Bar(name='Control', x= df_aux['city'], 
                     y= df_aux['time_mean'], error_y= dict(type='data', array= df_aux['time_std'])))

fig.update_layout(barmode='group')
fig.show()

### 2.3.4 Average delivery time and standard deviation by city and type of order.

In [24]:
df_aux = df2[['time_taken(min)', 'city', 'type_of_order']].groupby(['city', 'type_of_order']).agg({'time_taken(min)': ['mean', 'std']})
df_aux.columns = ['time_mean', 'time_std']
df_aux = df_aux.reset_index()

df_aux = df_aux.groupby('city').apply(lambda x: x.sort_values('time_mean', ascending= False)).reset_index(drop=True)
df_aux

,city,type_of_order,time_mean,time_std
0,Metropolitian,Meal,27.616383,9.214536
1,Metropolitian,Snack,27.468414,9.119676
2,Metropolitian,Drinks,27.322691,9.041655
3,Metropolitian,Buffet,27.299008,9.153107
4,Semi-Urban,Meal,50.300000,3.041665
5,Semi-Urban,Buffet,49.707317,2.731702
6,Semi-Urban,Drinks,49.625000,2.459347
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.560652,9.056348
9,Urban,Drinks,23.311977,8.927314


### 2.3.5 Average delivery time and standard deviation by city and traffic type.

**Obs: There is no 'Low' traffic in Semi-Urban cities**

In [54]:
df_aux = df2[['time_taken(min)', 'city', 'road_traffic_density']].groupby(['city', 'road_traffic_density']).agg({'time_taken(min)': ['mean', 'std']})
df_aux.columns = ['time_mean', 'time_std']
df_aux = df_aux.reset_index()
df_aux = df_aux.groupby('city').apply(lambda x: x.sort_values('time_mean', ascending= False)).reset_index(drop=True)
df_aux

,city,road_traffic_density,time_mean,time_std
0,Metropolitian,Jam,31.976991,9.476203
1,Metropolitian,High,28.140898,7.904645
2,Metropolitian,Medium,27.729966,8.308064
3,Metropolitian,Low,22.257675,6.794772
4,Semi-Urban,High,50.125000,2.629956
5,Semi-Urban,Jam,49.841270,2.717095
6,Semi-Urban,Medium,47.400000,2.011080
7,Urban,Jam,27.993164,10.078271
8,Urban,High,24.305335,8.494842
9,Urban,Medium,23.864962,8.393176


In [60]:
fig = px.sunburst(df_aux, path=['city', 'road_traffic_density'], values= 'time_mean',
                  color= 'time_std', color_continuous_scale='RdBu_r', 
                  color_continuous_midpoint= np.average(df_aux['time_std']))

fig.show()

### 2.3.6 Average delivery time during Festivals.

In [26]:
df_aux = df2[['festival', 'time_taken(min)']].groupby('festival').mean().reset_index()
df_aux = df_aux.rename(columns={'time_taken(min)': 'time_mean'})
df_aux

,festival,time_mean
0,No,26.162741
1,Yes,45.518607


In [27]:
df2.head()

,id,delivery_person_id,delivery_person_age,delivery_person_ratings,restaurant_latitude,restaurant_longitude,delivery_location_latitude,delivery_location_longitude,order_date,time_orderd,...,road_traffic_density,vehicle_condition,type_of_order,type_of_vehicle,multiple_deliveries,festival,city,time_taken(min),week_of_year,distance
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,High,2,Snack,motorcycle,0,No,Urban,24,11,3.025153
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,Jam,2,Snack,scooter,1,No,Metropolitian,33,12,20.183558
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,Low,0,Drinks,motorcycle,1,No,Urban,26,11,1.552760
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,14,7.790412
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,High,1,Snack,scooter,1,No,Metropolitian,30,12,6.210147
